This file is where I got up to with the `VideoMode` before I had to leave.

It's broken down into X parts:

Load in the hardware file (from Stephan @ QuTech), this is also on my github.

Setup the awg_map, this is similar to the gate_map discussed in `qtt_examples` when setting up the `VirtualDAC`. It's necessary to setup at least two channels for `VideoMode` as well as a marker channel (otherwise the 2D_scan function will not work).

Define the station that will be used in the `VideoMode`
This includes connecting to the relevant instruments, in this case the awg and fpga.
It's important that the instrument names are left as they are as the `VideoMode` class refers to these specific names in its operation, i.e. `fpga` and `virtual_awg` or `awg` you may want to check which one!

The awg and fpga are initialized with specific classes `virtual_awg` which is setup to work with the Tektronix_5014, and the `FPGA_ave` which is not setup to work with the Alazar_ATS_9360, but I have attempted to write a wrapper for that (which could be greatly improved upon!).

Once the instruments are loaded they need to be setup/armed for operation with `VideoMode`, the following steps are what I found to be some necessary steps. However these may not be completely correct!

Ideally when the `FPGA_ave` is optimised for the Alazar there will be no error messages on setup, i.e.

`WARNING:qcodes.instrument.base:Snapshot: Could not update parameter: ch1_cycle_num on fpga`

Also setting the `int_delay` and `int_time` manually as seen below will be overstepped. I think the whole point of the `VideoMode` is that it determines your waveform parameters (i.e. the sample rate and voltage range) and then setups up the FPGA accordingly to produce a map that achieves the same thing. I have not done this here, the current wrapper only allows for the read out of whatever is predefined in this code. So if you change the awg sweep parameters, it will have no effect on the fpga's acquisition (but it's meant to in the end product!).

When I last ran this code (28.09.2018) I received an error that the Kernel was dead, I am not sure if this is due to some incompatibility between qcodes and qtt or a bug in my wrapper. It seems to appear when trying to address the gui, perhaps my version of python was not appropriate!

-- Jacob CW

In [1]:
%matplotlib inline
import sys, os, tempfile
import numpy as np
import matplotlib.pyplot as plt
import qcodes
import qtt
import logging

In [2]:
from qcodes.instrument_drivers.tektronix.AWG5014 import Tektronix_AWG5014  # <--- The instrument driver
from qcodes.instrument_drivers.tektronix.AWGFileParser import parse_awg_file  # <--- A helper function
from qcodes import MatPlot, QtPlot
from qtt.instrument_drivers.virtual_awg import virtual_awg
from qtt.instrument_drivers.FPGA_ave import FPGA_ave
from qtt.measurements.videomode import VideoMode
from qtt.gui.live_plotting import livePlot
# download this file to directory
from hardware import hardware

# set data directory
datadir = os.path.join(tempfile.mkdtemp(), 'qdata')
qcodes.DataSet.default_io = qcodes.DiskIO(datadir)

logger = logging.getLogger()
logger.setLevel(logging.WARNING)

In [3]:
hardware = hardware('test')

# coaxes to gates: (awg,channel)    
awg_map = {'P1': (0,1), 'P2': (0,2), 'P3': (0,3), 'P4': (0,4),
            # markers to instruments: (awg,channel,marker)
            'm4i_mk': (0, 4, 1),  'awg_to_m4i_mk': (0,4,1),
            'awg_mk': (0, 4, 2)
          }

In [4]:
def initialize():

    logging.info('my station: initialize')
    
    # Connect to Tektronix AWG
    awg1 = Tektronix_AWG5014('awg1', 'TCPIP0::192.168.15.103::inst0::INSTR')
    
    # Create virtual awg
    awg = virtual_awg(name='virtual_awg', instruments=[awg1], awg_map=awg_map, hardware=hardware)
    
    # Create the ATS9360 instrument
    fpga = FPGA_ave(name='fpga', address='ASRL1::INSTR')
    
    logging.info('my station: all drivers have been loaded')

    #Creating the experimental station
    station = qcodes.Station(awg, fpga)

    logging.info('my station: initialization done')   
    return station

In [5]:
station = initialize()

fpga = station.fpga
virtual_awg = station.virtual_awg

Connected to: TEKTRONIX AWG5014C (serial:B051367, firmware:SCPI:99.0 FW:4.6.0.7) in 0.08s
FPGA internal clock is 50MHz, dividing it by 50, yields samp. freq. is 1000000 Hz


In [6]:
hardware.parameters['awg_to_P1'].set(1)
hardware.parameters['awg_to_P2'].set(1)
hardware.parameters['awg_to_P3'].set(1)
hardware.parameters['awg_to_P4'].set(1)

In [ ]:
mwindows = qtt.gui.live_plotting.setupMeasurementWindows(station, create_parameter_widget=False)
plotQ = mwindows['plotwindow']

In [ ]:
fpga.alazar_setup()

In [ ]:
fpga.setup_controller()

In [ ]:
fpga.set_int_delay(2e-5)
fpga.set_int_time(2e-6)

In [ ]:
fpga.setup_channel(channel='A', num_averages=1)

In [ ]:
sweepparams = ['P1', 'P2']
sweepranges = [0.5, 1.5]

vm = VideoMode(station, sweepparams=sweepparams, sweepranges=sweepranges, minstrument= [fpga, ['FPGA_ch1']], nplots = 1)